# Initial Conditions


In order to compute the dynamics in SEOBNRv5HM, we will need accurate quasi-circular initial conditions. these initial conditions need to be computed given the following waveform parameters:

- The total mass $M$ in solar masses
- The mass ratio $q$ given as $\tfrac{m_1}{m_2}$ s.t $q \geq 1$.
- The spins $S_1$, and $S_2$ of the binary in dimensionless units
- The starting gravitational wave frequency in Hz

First, we implement the systems of equations needed to be solved for the primitive variables and then build the initial conditions routine to get the primitive variables from the above inputs.

In [1]:
import sys,os#Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "Dynamics"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

# Step : Transform mass ratio to individual masses

We use the normalization choice $m_1 + m_2 \equiv M = 1$ to get

$$
m_1 = \frac{q}{1 + q}\\
m_2 = \frac{1}{1 + q}
$$

In [2]:
%%writefile -a $Ccodesdir/v5HM_Initial_Conditions.txt
m1 = q/(1 + q)
m2 = 1/(1 + q)

Writing Dynamics/v5HM_Initial_Conditions.txt


# Step : Incorporate the spins

We define the spins as 

$$
\chi_{1,2} = S_{1,2}
$$

Since in the spin-aligned model only the z-component is needed.

In [3]:
%%writefile -a $Ccodesdir/v5HM_Initial_Conditions.txt
chi1 = S1
chi2 = S2

Appending to Dynamics/v5HM_Initial_Conditions.txt


# Step : Convert the starting frequency to orbital frequency

Noting that the plane of the polarization of the gravitational wave "rotates at twice the orbital rate" (see the "Effects of passing" section of [this Wikipedia article](https://en.wikipedia.org/wiki/Gravitational_wave#Effects_of_passing)), the initial orbital frequency is
 
\begin{equation*}
    \omega = M \mathcal{M}_{\odot} \pi f.
\end{equation*}


In [4]:
%%writefile -a $Ccodesdir/v5HM_Initial_Conditions.txt
Msol = 4.925491025543575903411922162094833998e-6
Omega_start = M*Msol*sp.pi*f

Appending to Dynamics/v5HM_Initial_Conditions.txt


# Step : Get the initial guess for the radius and angular momentum

We use Kepler's law for circular orbits to get the following guess

$$
r_{\rm guess} = \omega^{-2/3}\\
p_{\phi,\rm guess} = r_{\rm guess}^{2}\omega = \omega^{-1/3}
$$

In [5]:
%%writefile -a $Ccodesdir/v5HM_Initial_Conditions.txt
r_guess = Omega_start**(-2/3)
pphi_guess = Omega_start**(-1/3)

Appending to Dynamics/v5HM_Initial_Conditions.txt


# Step : Compute the conservative initial conditions

In this step, we compute the purely conservative (i.e in the absence of radiation reaction) circular initial conditions.
The equations for a circular orbit with orbital frequency $\omega$ is given by:

$$
\dot{\phi} = \omega = \left.\frac{\partial H}{\partial p_{\phi}}\right|_{p_{r_*} = 0}\\
\dot{p}_{r} = -\left.\frac{\partial H}{\partial r}\right|_{p_{r_*} = 0}
$$

We solve the above system of non-linear system of equations.
Note: We are using a tolerance of `6e-12` as this is the choice pyseobnr makes.

In [6]:
%%writefile -a $Ccodesdir/v5HM_Initial_Conditions.txt
sol_cons_guess = np.array(r_guess)
params_cons = np.array([m1,m2,chi1,chi2,Omega_start])
sol_cons = root(IC_cons,sol_cons_guess,args = (params),tol = 6e-12)
r , pphi = sol_cons.x[0] , sol_cons.x[1]

Appending to Dynamics/v5HM_Initial_Conditions.txt


# Step : Compute the dissipative initial conditions

In this step, we find the value of $p_{r_*}$ using the ansatz:

$$
\frac{d r}{dt} \equiv \left[ \frac{ \frac{d p_{\phi}}{d t} }{ \frac{d p_{\phi}}{d r} } \right].
$$

Where, $\frac{d p_{\phi}}{d t} = \mathcal{F}_{\phi}$. In the quasi-circular limit $d\left(\frac{\partial H}{\partial r}\right) = 0$ we get:

$$
\frac{dl}{dr} = -\frac{ \frac{\partial^2 H}{\partial r^2} }{ \frac{\partial^2 H}{\partial r \partial p_{\phi}} }.
$$

Thus, we can find the initial value by $p_{r_*}$ by solving the following non-linear equation:

$$
\xi\frac{\partial H}{\partial p_{r_{*}}} = -\frac{ \mathcal{F}_{\phi} \frac{\partial^2 H}{\partial r \partial p_{\phi}} }{ \frac{\partial^2 H}{\partial r^2} }
$$

We use a guess bracket (i.e an interval the contains the root c.f [scipy.optimise documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root_scalar.html#scipy.optimize.root_scalar)) of $[-3\times10^{-2},0]$ that is consistent with pyseobnr. Additionally, pyseobnr uses an absolute tolerance of `1e-12`, and a relative tolerance of `1e-10`.


In [7]:
%%writefile -a $Ccodesdir/v5HM_Initial_Conditions.txt
params_diss = np.array([r,pphi,m1,m2,chi1,chi2])
prstar_bracket = [-3e-2,0]
prstar_sol = root_scalar(IC_diss, args = params, bracket = prstar_bracket, xtol = 1e-12, rtol = 1e-10)
prstar = prstar_sol.root

Appending to Dynamics/v5HM_Initial_Conditions.txt


# Step : Store the routine in a python file

In [9]:
with open(os.path.join(Ccodesdir,"v5HM_Initial_Conditions.py"),"w") as output:
    output.write("import numpy as np\n from scipy.optimize import root, root_scalar\nfrom ..Derivatives.v5HM_Initial_Conditions_Conservative import v5HM_unoptimized_IC_cons as IC_cons\nfrom ..Derivatives.v5HM_Initial_Conditions_Dissipative import v5HM_unoptimized_IC_diss as IC_diss\n")
    output.write("def v5HM_initial_conditions(M,q,S1,S2,f):\n")
    for line in list(open(os.path.join(Ccodesdir,"v5HM_Initial_Conditions.txt"),"r")):
        output.write("   %s \n"%line.rstrip())
    output.write("    return np.array([r,0.,prstar,pphi])")